## Decoder Transformer Tests

### Model - GPT Models


Aim : To prompt engineer, finetune GPT models 
Tools : OpenAI API (by google search)

How to use OpenAI API
- OpenAI API Documentation:  API Reference + Docs  https://platform.openai.com/docs/api-reference/introduction
- Sample code for different tasks: OpenAI Cookbook  https://cookbook.openai.com/

In [ ]:
import openai
import os
import openai
from dotenv import load_dotenv

load_dotenv()
key = os.getenv("OPENAI_API_KEY_2")

openai.api_key = key


#### Baseline 
##### 1- Naive
##### 2- Explained


##### Naive

In [ ]:
# Qualitative

post = "My friend got sick after vaccine, now my parents don't want to vaccinate me. What should i do?"



messages = [
        {"role": "system", "content": "You are an expert in inferring people's sentiment towards vaccination."},
        {"role": "user", "content": f"""

        Infer the reddit post author's sentiment towards vaccination as positive or negative or neutral \n post: {post}

"""}
]

response = openai.chat.completions.create(
        model="gpt-4.1-2025-04-14",
        messages=messages,
        temperature=0.0
)

print(response.choices[0].message.content)

In [ ]:
# Quantitative

from tqdm import tqdm
import pandas as pd

df = pd.read_csv(r"..\input_data\input_output_test.csv")

predictions = []


for post in tqdm(df["input"]):
        messages = [
                {"role": "system", "content": "You are an expert in inferring people's sentiment towards vaccination."},
                {"role": "user", "content": f"""
        
         Infer the reddit post author's sentiment towards vaccination as positive or negative or neutral \n post: {post}
     
        """}
        ]
        
        response = openai.chat.completions.create(
                model="gpt-4.1-2025-04-14",
                messages=messages,
                temperature=0.0
        )
        
        predictions.append(response.choices[0].message.content)

In [ ]:
# Cleaning the Quantiative Results

import re

cleaned = []
for text in predictions:
    match = re.search(r'\b(Positive|Negative|Neutral)\b', text, re.IGNORECASE)
    if match:
        cleaned.append(match.group(1).capitalize())  # normalize case
    else:
        cleaned.append("Unknown")

print(len(cleaned))
print(cleaned)


In [ ]:
# Saving the Quantiative Results

df["pred"] = cleaned


output_path = r"..\prediction\input_output_test_prediction_naive_baseline_gpt4.1_4.csv"
df.to_csv(output_path, index=False)

##### Explained

In [ ]:
# Qualitative

post = "My friend got sick after vaccine, now my parents don't want to vaccinate me. What should i do?"




messages = [
        {"role": "system", "content": """The positive stance includes posts that support vaccination, show genuine interest in taking it, encourage others, discourage negative sentiment towards vaccination, and counter the anti-vaccine community.
        The negative stance includes posts that oppose vaccination, lack interest in getting vaccinated, discourage others from taking the vaccine, and counter the vaccine community.
        The neutral stance consists of posts that are not clearly positive or negative. Posts can share experiences (can be painful), but the intention is not to discourage vaccination."""},
        
        {"role": "user", "content": f"""
        You are an expert in vaccine stance inference. Using the above information, infer the stance of the following Reddit post \n post: {post} 
        """}
        ]

response = openai.chat.completions.create(
        model="gpt-4.1-2025-04-14",
        messages=messages,
        temperature=0.0
)

print(response.choices[0].message.content)

In [ ]:
# Quantiative 

from tqdm import tqdm
import pandas as pd

df = pd.read_csv(r"..\input_data\input_output_test.csv")

predictions1 = []


for post in tqdm(df["input"]):
        messages = [
        {"role": "system", "content": """The positive stance includes posts that support vaccination, show genuine interest in taking it, encourage others, discourage negative sentiment towards vaccination, and counter the anti-vaccine community.
        The negative stance includes posts that oppose vaccination, lack interest in getting vaccinated, discourage others from taking the vaccine, and counter the vaccine community.
        The neutral stance consists of posts that are not clearly positive or negative. Posts can share experiences (can be painful), but the intention is not to discourage vaccination."""},
        {"role": "user", "content": f"""With the above information, infer the reddit post author's sentiment towards vaccination. \n post: {post} 
        """}
        ]
        
        response = openai.chat.completions.create(
                model="gpt-4.1-2025-04-14",
                messages=messages,
                temperature=0.0
        )
        
        predictions1.append(response.choices[0].message.content)

print(len(predictions1))

In [ ]:
# Cleaning the Quantiative Results

import re

cleaned = []
for text in predictions1:
    match = re.search(r'\b(Positive|Negative|Neutral)\b', text, re.IGNORECASE)
    if match:
        cleaned.append(match.group(1).capitalize())  # normalize case
    else:
        cleaned.append("Unknown")

print(len(cleaned))
print(cleaned)


In [ ]:
# Saving the Quantiative Results

df["pred"] = cleaned


output_path = r"..\prediction\filename.csv"
df.to_csv(output_path, index=False)

#### Role-Based Incremental Coaching


Role Based knowledge Generation 

In [ ]:
messages = [
		{"role": "system", "content": """You are an expert in inferring a post’s stance towards vaccination. 
       			Vaccine Stance is of 3 types - 
			Positive: Posts that support vaccination, show genuine interest in taking it, encourage others, discourage negative sentiment towards vaccination, and counter the anti-vaccine community. 
			Negative: Posts that oppose vaccination, lack interest in getting vaccinated, discourage others from taking the vaccine, and counter the vaccine community. 
			Neutral: Posts that are not clearly Positive or Negative. It could be sharing experience (can be painful) but the intention is not to discourage vaccination."""},
		{"role": "user", "content": """ List some common linguistic cues present in textual Reddit posts that contain vaccine sentiment that can help infer the sentiment of the post."""}
]

response = openai.chat.completions.create(
    model="gpt-4.1-2025-04-14",
    messages=messages,
    temperature=0.0
)

messages.append({"role": "assistant", "content": response.choices[0].message.content})
print(response.choices[0].message.content)


Incremental Coaching

In [ ]:
# Quantitative 

from tqdm import tqdm
import pandas as pd

df = pd.read_csv(r"..\input_data\input_output_test.csv")

predictions2 = []


for post in tqdm(df["input"]):

    messages_temp = messages.copy()
    
 

    prompt_simplified = f""" post: {post} Simplify the language of the post without changing its meaning. Capture the key points and express them in a clear and concise manner """


    messages_temp.append({"role": "user", "content": f"{prompt_simplified}"})

    response = openai.chat.completions.create(
        model="gpt-4.1-2025-04-14",
        messages=messages_temp,
        temperature=0.0
    )




    prompt_sentiment = "Infer the post author's sentiment towards vaccination, with all the information. (Return only the sentiment (positive/negative/neutral))"

    messages_temp.append({"role": "assistant", "content": response.choices[0].message.content})
    messages_temp.append({"role": "user", "content": f"{prompt_sentiment}"})

    response = openai.chat.completions.create(
        model="gpt-4.1-2025-04-14",
        messages=messages_temp,
        temperature=0.0
    )

    predictions2.append(response.choices[0].message.content)


print(len(predictions2))

In [ ]:
print(predictions2)

In [ ]:
# Saving the Quantitative Results

df["pred"] = predictions2


output_path = r"..\prediction\filename.csv"
df.to_csv(output_path, index=False)

#### DSPy

Aim : To optimise the prompt engineered & finetuned GPT Model Performance using DSPy.   
DSpy Documentation: https://dspy.ai/

##### 1. DSPy Simple Module (Module Used - Chain of Thought) 
Selected after reading all modules' desciptions, involves a reasoning step, can improve performance.

In [ ]:
import dspy 

dspy.configure(lm = dspy.LM("openai/gpt-4.1-2025-04-14", temperature = 0, api_key = key, cache=True))

In [ ]:
# Signature Definition
from typing import Literal

class vaccine_stance_signature(dspy.Signature):
        """
        Stance towards vaccination is of 3 types: 
        positive: Posts that support vaccination, show genuine interest in taking it, encourage others, discourage negative sentiment towards vaccination, and counter the anti-vaccine community.
        negative: Posts that oppose vaccination, lack interest in getting vaccinated, discourage others from taking the vaccine, and counter the vaccine community.
        neutral: Posts that are not clearly positive or negative. Posts can share experiences (can be painful), but the intention is not to discourage vaccination.
        """

        post : str = dspy.InputField(desc = "Social media post from which the poster's stance, towards vaccination, needs to be inferred.")
        stance : Literal["positive", "negative", "neutral"]  = dspy.OutputField(desc = "One worded stance label.")


In [ ]:
simple_cot = dspy.ChainOfThought(vaccine_stance_signature)
simple_cot

In [ ]:
stance = simple_cot(post = "people always hate vaccines")
stance

In [ ]:
from tqdm import tqdm
import pandas as pd

df = pd.read_csv(r"..\..\..\..\input_data\input_output_test.csv")  


predictions = []

for post in tqdm(df["input"]):
        stance = simple_cot(post = post)
        predictions.append(stance.stance)
        

print(len(predictions))

##### 2. dspy.BootstrapFewShotWithRandomSearch modified

In [ ]:
import random
import numpy as np
from sklearn.metrics import f1_score

class BootstrapFewShotWithRandomSearch_modified:
    def __init__(self, base_cot_module, t=16, n=4, k=16, temperature=0):
        
        self.base = base_cot_module
        self.t = t
        self.n = n
        self.k = k
        self.temperature = temperature

    def fit(self, train_data, valid_data):

        best_f1 = -1
        best_examples = None

        for trial in range(self.t):
            candidate_idxs = random.sample(range(len(train_data)), self.n)
            candidate_subset = [train_data[i] for i in candidate_idxs]

            y_true = []
            y_pred = []

            for text, gold_label in valid_data:
                result = self.base.predict(text, temperature=self.temperature)
                y_true.append(gold_label)
                y_pred.append(result["answer"])

            trial_f1 = f1_score(y_true, y_pred, average="macro")


            solved_examples = []

            for text, gold_label in train_data:
                result = self.base.predict(text, temperature=self.temperature)
                if result["answer"] == gold_label:
                    solved_examples.append({
                        "input": text,
                        "label": gold_label,
                        "reasoning": result["reasoning"]
                    })


            if len(solved_examples) >= self.k:
                solved_examples = random.sample(solved_examples, self.k)


            few_shot_examples = {
                "prompt_examples": candidate_subset,
                "cot_examples": solved_examples
            }


            if trial_f1 > best_f1:
                best_f1 = trial_f1
                best_examples = few_shot_examples


        self.best_config = best_examples
        self.best_f1 = best_f1
        return best_examples



In [ ]:
optimizer = BootstrapFewShotWithRandomSearch_modified(
    base_cot_module=simple_cot,   # your DSPy CoT module
    t=16,
    n=4,
    k=16,
    temperature=0
)

train_data = pd.read_csv(r"..\..\..\..\input_data\input_output_train_shuffled_90.csv").to_dict(orient="records")
valid_data = pd.read_csv(r"..\..\..\..\input_data\input_output_validation_shuffled_360.csv").to_dict(orient="records")

best_few_shot = optimizer.fit(train_data, valid_data)

print("Best macro-F1:", optimizer.best_f1)
print("Selected few-shot examples:", best_few_shot)  #saved at ../input_data/demos.json


#### Finetunening

##### Finetuning Code

In [ ]:
# File Upload
# File Only Accepted in JSONL Format

import requests
import os

# Your OpenAI API Key
OPENAI_API_KEY = key

# API endpoint
url = 'https://api.openai.com/v1/files'

# Headers with authorization
headers = {
    'Authorization': f'Bearer {OPENAI_API_KEY}'
}


file_path = r"..\input_data\vaccine_sentiment_test_formatted.jsonl"

# Ensure the file exists
if not os.path.isfile(file_path):
    raise FileNotFoundError(f"File not found: {file_path}")

# Prepare data and file for upload
data = {
    'purpose': 'fine-tune'
}

files = {
    'file': (
        os.path.basename(file_path), 
        open(file_path, 'rb'), 
        'application/jsonl'
    )
}

# Make the request
response = requests.post(url, headers=headers, data=data, files=files)

# Close the file after upload
files['file'][1].close()

# Display response
print(response.status_code)
print(response.json())


In [ ]:
# Fine-tuning job Start 

import os
import requests


API_KEY = key
if not API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable")


training_file_id = "training-file-name"     # From the previous file upload response (training file - 450*3)
validation_file_id ="validation-file-name"    # From the previous file upload response (test file - 50*2 + 52)

MODEL_NAME = "gpt-4.1-2025-04-14"

url = "https://api.openai.com/v1/fine_tuning/jobs"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}
payload = {
    "training_file": training_file_id,
    "validation_file": validation_file_id,
    "model": MODEL_NAME ,
        "hyperparameters": {
        "n_epochs": 3, 
        # rest are default values
    }
}

response = requests.post(url, headers=headers, json=payload)
print("Status:", response.status_code)
print(response.json())


In [ ]:
# Cancel a fine-tuning job already running

job_id = "ftjobID"  
url = f"https://api.openai.com/v1/fine_tuning/jobs/{job_id}/cancel"

response = requests.post(url, headers=headers)
print("Cancel status:", response.status_code)
print(response.json())


In [ ]:
# Check the status of the fine-tuning job

import os
import requests


API_KEY = key
if not API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable")


job_id = "ftjob-ID"


url = f"https://api.openai.com/v1/fine_tuning/jobs/{job_id}"
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(url, headers=headers)


print("HTTP", response.status_code)
print(response.json())

In [ ]:
# Print the events of a fine-tuning job events to check the loss on validation file 

client = openai.OpenAI(
    api_key=key)

response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

In [ ]:
# Save the fine-tuning job details

import os
import requests


API_KEY = key
if not API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable")


file_id = "file-jobID"             
output_path = r"..\prediction\filename.csv"        


url = f"https://api.openai.com/v1/files/{file_id}/content"
headers = {"Authorization": f"Bearer {API_KEY}"}

with requests.get(url, headers=headers, stream=True) as resp:
    resp.raise_for_status()  
    with open(output_path, "wb") as f:
        for chunk in resp.iter_content(chunk_size=8192):
            f.write(chunk)

print(f"Saved result to {output_path}")

##### Test Code

In [ ]:
# Qualitative Test

import os
import requests
import pandas as pd


API_KEY = key
if not API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable")


MODEL_NAME = "fine-tuned model id" # Obtained after fine-tuning from the saved details
url = "https://api.openai.com/v1/responses"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

post  = ""



payload = {
    "model": MODEL_NAME,
    "input": post, 
    "temperature": 0
    }
resp = requests.post(url, headers=headers, json=payload)
resp.raise_for_status()


data = resp.json()

model_output = data.get("response") or data.get("choices", [{}])[0].get("text", "")

print("Output:", data["output"][0]["content"][0]["text"])

In [ ]:
# Quantitative Test

import os
import time
import requests
import pandas as pd
from tqdm import tqdm


API_KEY = key
if not API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY variable")


input_path  = r"..\input_data\input_output_test.csv"  
output_path = r"..\prediction\filename.csv"
MODEL_NAME  = "fine-tuned model id" # Obtained after fine-tuning from the saved details


df = pd.read_csv(input_path, encoding="utf-8")


raw_responses    = []


url = "https://api.openai.com/v1/responses"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

for _, row in tqdm(df.iterrows()):
    user_input = row.get("input")  
    
    resp = requests.post(
        url,
        headers=headers,
        json={"model": MODEL_NAME, "input": user_input, "temperature": 0}
    )
    resp.raise_for_status()
    data = resp.json()
    
    
    raw = data["output"][0]["content"][0]["text"].strip().lower()
    raw_responses.append(raw)
    


df["pred"]    = raw_responses

df.to_csv(output_path, index=False, encoding="utf-8")

print(f"Done! Responses written to {output_path}")

In [ ]:
# Checkpoint details retrieval

import requests

api_key = key
job_id = "ft-jobID"

url = f"https://api.openai.com/v1/fine_tuning/jobs/{job_id}/checkpoints"

headers = {
    "Authorization": f"Bearer {api_key}",
}

response = requests.get(url, headers=headers)


print(response.json())


#### F1 Score

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

df = pd.read_csv(r"..\prediction\filename.csv")

# Map columns
df['y_true'] = df['output']
df['y_pred'] = df['pred'].str.lower().map({'positive': 1, 'negative': -1, 'neutral': 0})

# Overall accuracy
overall_accuracy = (df['y_true'] == df['y_pred']).mean()
print(f"Overall accuracy: {overall_accuracy:.2%}")

# Precision / Recall / F1
report = classification_report(
    df['y_true'], 
    df['y_pred'], 
    target_names=['negative', 'neutral', 'positive']
)
print("\nClassification Report:\n", report)